In [1]:
import gradio as gr
import numpy as np
from PIL import Image
from typing import Tuple

# Assuming we have our ArtAnalyzer class defined
from artclass_class import ArtAnalyzer

def load_model():
    # Initialize our analyzer with the correct style order from your training
    style_labels = [
    "naive_art",
    "baroque",
    "rococo",
    "romanticism", 
    "art_deco",
    "american_realism",
    "art_nouveau",
    "expressionism",
    "post_impressionism",
    "high_renaissance",
    "cubism",
    "abstract_expressionism",
    "art_informel",
    "mannerism",
    "northern_renaissance",
    "surrealism",
    "symbolism", 
    "early_renaissance",
    "modernism",
    "neo_romantic",
    "ukiyo_e",
    "impressionism",
    "pop_art",
    "fauvism",
    "neoclassicism",
    "minimalism"
]
    
    return ArtAnalyzer(
        cnn_model_path="path/to/your/model.pkl",
        style_labels=style_labels
    )

def process_and_analyze(image: np.ndarray, question: str) -> Tuple[str, str]:
    """
    Process image and use ArtAnalyzer's methods for analysis
    """
    # Preprocess image
    img = Image.fromarray(image)
    img = img.resize((256, 256))  # adjust size to match your model's requirements
    img_array = np.array(img) / 255.0  # basic normalization
    
    # Use analyzer's analyze_image method
    predictions = analyzer.analyze_image(img_array)
    
    # Format style predictions
    style_output = "Predicted Styles:\n"
    sorted_predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
    for style, conf in sorted_predictions[:3]:
        style_output += f"{style}: {conf:.2%}\n"
    
    # Use analyzer's question answering method
    response = analyzer.answer_question(question, predictions)
    
    return style_output, response

# Initialize analyzer
analyzer = load_model()

# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Art Style Analyzer")
    
    with gr.Row():
        # Input components
        image_input = gr.Image()
        question_input = gr.Textbox(
            label="Ask a question about the artwork",
            placeholder="What techniques are used in this style?"
        )
    
    with gr.Row():
        # Output components
        style_output = gr.Textbox(label="Style Predictions")
        answer_output = gr.Textbox(label="Answer")
    
    # Button to trigger analysis
    analyze_btn = gr.Button("Analyze Artwork")
    
    # Set up event handler
    analyze_btn.click(
        fn=process_and_analyze,
        inputs=[image_input, question_input],
        outputs=[style_output, answer_output]
    )

# Launch the interface
if __name__ == "__main__":
    demo.launch()

ModuleNotFoundError: No module named 'artclass_class'